In [0]:
import tensorflow as tf
print(tf.__version__)

2.0.0-rc0


In [0]:
import tensorflow_datasets as tfdf

In [0]:
#Imdm_dataet
imdb,info = tfdf.load("imdb_reviews",with_info = True,as_supervised = True)

In [0]:
#Split the dataset into train and test dataset
import numpy as np
train_data,test_data = imdb['train'],imdb['test']

In [0]:
#Converting the datasets into numpy array
training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []

for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

In [0]:
#data preprocessing
vocab_size = 10000
embedding_dim  = 16
max_length = 120
trunc_type = 'post'
oov_token = "<00V>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size , oov_token = oov_token)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen = max_length,truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen = max_length)

In [0]:
#Converting the label dataset to numpy arrays
testing_labels_final = np.array(testing_labels)
training_labels_final = np.array(training_labels)

In [0]:
#To reverse the word index syntax from key,value pair to value,key pair for training
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i,'?') for i in text])
print(decode_review(padded[1]))
print(training_sentences[1])

In [0]:
#define the model for neural network
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length = max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(1,activation = 'sigmoid')
])

In [0]:
#Compile the model and printing the summary
model.compile(loss='binary_crossentropy',optimizer='adam',metrics = ['accuracy'])
model.summary()

In [0]:
#Define the checkpoints during training data
from keras.callbacks import ModelCheckpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [53]:
#training the data
model.fit(padded,training_labels_final,epochs = 20,validation_data = (testing_padded,testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 5s 196us/sample - loss: 0.4952 - accuracy: 0.7397 - val_loss: 0.3497 - val_accuracy: 0.8450
Epoch 2/20
25000/25000 [==============================] - 4s 174us/sample - loss: 0.2438 - accuracy: 0.9050 - val_loss: 0.3741 - val_accuracy: 0.8357
Epoch 3/20
25000/25000 [==============================] - 4s 173us/sample - loss: 0.0974 - accuracy: 0.9733 - val_loss: 0.4451 - val_accuracy: 0.8268
Epoch 4/20
25000/25000 [==============================] - 4s 173us/sample - loss: 0.0253 - accuracy: 0.9968 - val_loss: 0.5290 - val_accuracy: 0.8215
Epoch 5/20
25000/25000 [==============================] - 4s 176us/sample - loss: 0.0058 - accuracy: 0.9996 - val_loss: 0.5995 - val_accuracy: 0.8228
Epoch 6/20
25000/25000 [==============================] - 4s 172us/sample - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.6482 - val_accuracy: 0.8240
Epoch 7/20
25000/25000 [==========================

In [54]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [0]:
#Opening the vecs and meta files for Interpretaion of data
import io
out_v = io.open('vecs.tsv','w',encoding = 'utf-8')
out_m = io.open('meta.tsv','w',encoding = 'utf-8')

for word_num in range(1,vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()